<a href="https://colab.research.google.com/github/YunhZhang/CS541/blob/HW5/Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import numpy as np
from tqdm import trange
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Dataloder


In [5]:
x = np.load("faces.npy")/255
y = np.load("ages.npy")
x = torch.tensor(x).to(device)
y = torch.tensor(y).to(device)

ValueError: ignored

In [27]:
# def load_data():
#   x = np.load("faces.npy")/255
#   y = np.load("ages.npy")
#   x = torch.tensor(x).to(device)
#   y = torch.tensor(y).to(device)
#   return x,y

class FaceLoader(torch.utils.data.Dataset):
  def __init__(self, faces, transform=None, time_num=1):
    super(FaceLoader, self).__init__()

    n,l,w = faces.shape
    faces = faces.unsqueeze_(2)
    faces = faces.expand(n,l,3,w)
    self.faces = faces
    self.transform = transform
    self.time_num = time_num

  def __len__(self):
    return self.faces.size(0)

  def __fetitem__(self, item):
    face = self.transform(self.faces[item])
    return face
    
class AgeLoader(torch.utils.data.Dataset):
  def __init__(self, ages, time_num=1):
    super(AgeLoader, self).__init__()

    self.ages = ages
    self.time_num = time_num

  def __len__(self):
    return self.ages.size(0)

  def __fetitem__(self, item):
    age = self.ages[item]
    return age

transform = torchvision.transforms.Compose([
                                torchvision.transforms.ToPILImage(),
                                torchvision.transforms.Resize((224,224)),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])
                              ])

face = FaceLoader(faces=x, transform=transform)
age = AgeLoader(ages=y)


In [28]:
N = len(age)
test_ratio = 0.2
valid_ratio = 0.2

test_N = int(N*test_ratio)
def split_tensor(tensor, ratio):
  length = len(tensor)
  test_num = int(length*ratio)
  train, test = random_split(tensor, lengths=[length-test_num,test_num],generator=torch.Generator().manual_seed(42))
  return train, test

face_train, face_test = split_tensor(face,test_ratio)
face_train, face_valid = split_tensor(face_train,valid_ratio)

age_train, age_test = split_tensor(age,test_ratio)
age_train, age_valid = split_tensor(age_train,valid_ratio)

batchTrain = 10
batchValid = 8
batchTest = 8
params_train = {'batch_size':batchTrain,"shuffle":True,'num_workers':0}
params_valid = {'batch_size':batchValid,"shuffle":False,'num_workers':0}
params_test = {'batch_size':batchTest,"shuffle":False,'num_workers':0}

train_faces = DataLoader(face_train, **params_train)
train_ages = DataLoader(age_train, **params_train)

valid_faces = DataLoader(face_valid, **params_valid)
valid_ages = DataLoader(age_valid, **params_valid)

test_faces = DataLoader(face_test, **params_test)
test_ages = DataLoader(age_test, **params_test)

# NN architecture

In [32]:
model = torchvision.models.resnet50(weights='DEFAULT').to(device)
model.fc2 = nn.Linear(1000,1)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [30]:
learning_rate = 0.0001
beta = (0.995,0.998)
eps = 1e-10
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=beta, eps=eps)

# Train

In [33]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

criterion = RMSELoss()

In [21]:
train_loss = []
valid_loss = []
best_loss = 100
Epoch = 10

In [ ]:
for epoch in trange(Epoch):
  n = 0
  dynamic_loss = 0
  for i, (faces_, ages_) in enumerate(zip(train_faces,train_ages),0):
    faces = faces_.to(dtypr=torch.float32).to(device)
    ages = ages_.to(dtypr=torch.float32).to(device)
    output = model(faces)
    loss = criterion(output,ages)
    
    n += faces.shape[0]
    dynamic_loss += loss.item()

    optimizeer.zero_grad()
    loss.backward()
    optimizer.step
  train_loss.append(dynamic_loss/n)

  for i_val, (faces_v_, ages_v_) in enumerate(zip(valid_faces,valid_ages),0):
    faces_v = faces_v_.to(dtypr=torch.float32).to(device)
    ages_v = ages_v_.to(dtypr=torch.float32).to(device)


In [35]:
train_faces.device()

AttributeError: ignored

In [8]:
x = np.load("faces.npy")

In [9]:
x.shape

(7500, 48, 48)

In [14]:
x.device

device(type='cuda', index=0)

In [15]:
x_ = x.unsqueeze_(2)

In [16]:
x_.shape

torch.Size([7500, 48, 1, 48])

In [17]:
x_ = x_.expand(7500,48,3,48)

In [18]:
x_.shape

torch.Size([7500, 48, 3, 48])

In [19]:
x_[0,:,0,:]-x_[0,:,1,:]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')